# Preamble

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../SatLib')
import satbox as sb

In [3]:
import numpy as np
from poliastro.bodies import Earth
import astropy.units as u
from astropy.time import Time, TimeDelta

# Define Walker 

In [4]:
i = 60 * u.deg
t = 4
p = 2
f = 0
alt = 550 * u.km
walker = sb.Constellation.from_walker(i,t,p,f,alt)

/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


# Define Ground Station

In [5]:
gs_lat = 39 * u.deg
gs_lon = 127 * u.deg
gs_h = 0 * u.m
gs = sb.GroundLoc(gs_lon, gs_lat, gs_h)

# Define RGT Drift Maneuver

In [6]:
r_drift = 650*u.km #Drift orbit at 650 km

# Calculate Burn schedule

In [7]:

schedDict = walker.gen_GOM_2_RGT_scheds(r_drift, gs) #Function that generates burn schedules given an r_drift and a ground target location in lat/lon for each satellite

#From the selection of all burn schedules, find the ones with the least drift time
sats2Maneuver, driftTimes, sched = walker.get_ascending_descending_per_plane(schedDict) #Assumes two satellite per plane will get there


# Propagate satellites

In [8]:
t2propagate = 1*u.day
tStep = 30 * u.s
verbose = True
walkerSim = sb.SimConstellation(walker, t2propagate, tStep, verbose = verbose)
walkerSim.propagate(method=None, select_sched_sats = sats2Maneuver, verbose=verbose)

maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 0
maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 1
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 2
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 3


In [9]:
#Here is the consetllation you want to plot
walkerSim.constellation.planes[0].sats[0].__dict__

# This is each of the segments in the burn trajectory
walkerSim.constellation.planes[0].sats[0].cartesianRepSegments

[<CartesianRepresentation (x, y, z) in km
     [( 6928.1366    , -8.48452031e-13, -1.46956203e-12),
      ( 6924.40003006,  1.16276547e+02,  2.01396887e+02),
      ( 6913.1948919 ,  2.32427683e+02,  4.02576556e+02),
      ( 6894.53489113,  3.48328186e+02,  6.03322115e+02),
      ( 6868.4428391 ,  4.63853213e+02,  8.03417332e+02),
      ( 6834.95060608,  5.78878489e+02,  1.00264695e+03),
      ( 6794.09905591,  6.93280491e+02,  1.20079703e+03),
      ( 6745.93796261,  8.06936634e+02,  1.39765525e+03),
      ( 6690.525909  ,  9.19725452e+02,  1.59301121e+03),
      ( 6627.93016803,  1.03152678e+03,  1.78665678e+03),
      ( 6558.22656718,  1.14222191e+03,  1.97838638e+03),
      ( 6481.4993365 ,  1.25169379e+03,  2.16799724e+03),
      ( 6397.84094096,  1.35982718e+03,  2.35528976e+03),
      ( 6307.35189772,  1.46650878e+03,  2.54006772e+03),
      ( 6210.14057918,  1.57162745e+03,  2.72213859e+03),
      ( 6106.3230024 ,  1.67507428e+03,  2.90131376e+03),
      ( 5996.02260593,  1.7767

In [10]:
#This is all the segments in one array
walkerSim.constellation.planes[0].sats[0].coordECI

<CartesianRepresentation (x, y, z) in km
    [(6928.1366    , -8.48452031e-13, -1.46956203e-12),
     (6924.40003006,  1.16276547e+02,  2.01396887e+02),
     (6913.1948919 ,  2.32427683e+02,  4.02576556e+02), ...,
     (5540.37681322,  2.04841563e+03,  3.54377296e+03),
     (5401.77836979,  2.13908314e+03,  3.70073410e+03),
     (5257.25178368,  2.22740313e+03,  3.85363389e+03)]
 (has differentials w.r.t.: 's')>